In [1]:
import os

In [2]:
%pwd

'd:\\DeepLearningE2E\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DeepLearningE2E'

#### Update Entity


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#### Update the configuration manager in src config


In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

#### Update the components

In [7]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(
                f"Downloading dataset from {dataset_url} into file {zip_download_dir}"
            )

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(
                f"Downloaded dataset from {dataset_url} into file {zip_download_dir}"
            )

        except Exception as e:
            raise e

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

#### Update the pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-31 16:06:23,442: INFO: common] Loaded yaml file from config\config.yaml
[2024-01-31 16:06:23,443: INFO: common] Loaded yaml file from params.yaml
[2024-01-31 16:06:23,444: INFO: common] Created directory artifacts
[2024-01-31 16:06:23,445: INFO: common] Created directory artifacts/data_ingestion
[2024-01-31 16:06:23,446: INFO: 391669600] Downloading dataset from https://drive.google.com/file/d/1abrn4KQuJ7i0BXkzgZmOdYiocgEQ3nmY/view?usp=drive_link into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1abrn4KQuJ7i0BXkzgZmOdYiocgEQ3nmY
From (redirected): https://drive.google.com/uc?export=download&id=1abrn4KQuJ7i0BXkzgZmOdYiocgEQ3nmY&confirm=t&uuid=358e633d-08b2-4dc1-be63-1e27478bd86e
To: d:\DeepLearningE2E\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:02<00:00, 55.3MB/s] 

[2024-01-31 16:06:26,660: INFO: 391669600] Downloaded dataset from https://drive.google.com/file/d/1abrn4KQuJ7i0BXkzgZmOdYiocgEQ3nmY/view?usp=drive_link into file artifacts/data_ingestion/data.zip
